<a href="https://colab.research.google.com/github/Felipe-Oliveira11/Fraud-Detection-ML/blob/master/Detec%C3%A7%C3%A3o_de_Fraude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detecção de Fraude de Cartões 


Neste projeto vamos aplicar Machine learning para Detecção de fraude em Cartões de crédito, um problema muito grande que instituições financeiras e Fintechs tem diariamente, que é identificar se uma transação é uma fraude ou não, é uma tarefa díficil e extremamente delicada. É de extrema importância que as empresas emissoras de cartão de crédito estejam preparadas para este tipo de crime, monitorando constantemente o comportamento dos cartões. 
<hr>

<p align=center>
<img src="https://miro.medium.com/max/1000/0*_6WEDnZubsQfTMlY.png" width="70%"></p>

https://ai-journey.com/wp-content/uploads/2019/06/fraud-EMV-chip-credit-card.jpg



Muitas empresas já tem investido fortemente na Inteligência Artificial, para solucionar problemas financeiros e também criar produtos que minimizam riscos e maximizam lucros. 

Há diversos cases na indústria, sobre a importância que tem a Inteligência Artificial para as empresas que lidam com dados financeiros, que necessitam de escalabilidade e velocidade em aprovações de transações feitas por cartões de crédito, é um desafio muito maior que apenas criar um modelo de Machine learning, mas também permitir um alto desempenho dessas plataformas. 


Com o crescimento da necessidade, de se ter um sistema robusto que consiga lhe dizer se uma compra foi ou não fraude, as oportunidades de se aplicar Inteligência Artificial são cada vez mais necessárias, e vem se tornando imprescindível fazer uso da IA para combater um problema tão grande.


<p align=center>
<img src="https://news.mit.edu/sites/mit.edu.newsoffice/files/styles/news_article_image_top_slideshow/public/images/2018/MIT-Fraud-Detection-PRESS_0.jpg?itok=laiU-5nR" width="60%"></p>


https://www.eastwestbank.com/ReachFurther/NewsArticleStore/519/Credit-card-fraud-top.jpg


<hr>

### Solução 

O objetivo é fazer o uso do Machine learning para antecipar uma fraude, reduzindo posteriormente os números de fraudes que acabam se concretizando e que não são identificadas a tempo, é uma tarefa que vai exigir muitos experimentos, técnicas e algoritmos que melhor performam em cima dos dados que temos, além de o modelo deve ter o mesmo nível de performance quando inserido em produção.  

<hr>
<br>

### *Sobre* o dataset

Os conjuntos de dados contêm transações realizadas com cartões de crédito em setembro de 2013 por portadores de cartões europeus.
Este conjunto de dados <b>apresenta transações que ocorreram em dois dias </b>, nas quais temos 492 fraudes em 284.807 transações. O conjunto de dados é altamente desequilibrado, a classe positiva (fraudes) representa 0,172% de todas as transações.

<br>

Ele contém apenas variáveis ​​de entrada numéricas que são o resultado de uma transformação PCA. Infelizmente, devido a <b> problemas de confidencialidade </b>, não podemos fornecer os recursos originais e mais informações básicas sobre os dados.


Features V1, V2,… V28 são os principais componentes obtidos com o PCA, as únicas features que não foram transformadas com o PCA são 'Time' (Tempo) e 'Amount' (Valor da transação).

<br> 


* A feature 'Time'(Tempo) contém os segundos decorridos entre cada transação e a primeira transação no conjunto de dados.

*  A feature 'Amount' é o valor da transação. 

* A feature 'Class' é a variável de resposta e assume o valor 1 em caso de fraude e 0 em caso contrário.

<br>
<b>
 0: Transação normal  |
 1: Transação fraudulenta
 </b>

<br>
<hr>

In [0]:
# pacotes 
!pip install shap 
!pip install yellowbrick
!pip install imblearn

In [0]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from tqdm import tqdm_notebook

from sklearn.metrics import classification_report, recall_score, precision_score ,average_precision_score, plot_precision_recall_curve
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.impute import SimpleImputer
from yellowbrick.classifier import PrecisionRecallCurve 
from sklearn.model_selection import train_test_split, cross_validate ,KFold, StratifiedShuffleSplit, StratifiedKFold
from sklearn.preprocessing import StandardScaler, PowerTransformer, QuantileTransformer, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier 
from lightgbm import LGBMClassifier
import shap
shap.initjs()

%matplotlib inline 
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [0]:
# importando dados 
path = '/content/drive/My Drive/ML_Notebooks/Detecção de Fraude /creditcard.csv' 
credit = pd.read_csv(path)
credit.head()

In [0]:
print('Linhas: {} | Colunas: {} '.format(credit.shape[0], credit.shape[1]))

In [0]:
# Estatística descritiva
credit.describe()

In [0]:
# Classes
sns.countplot(x=credit['Class'], palette='coolwarm')
print('Normal: {} |  Fraude: {}'.format(credit[credit['Class']==0].shape[0] , credit[credit['Class']==1].shape[0]))

De inicío fica claro o desbalanceamento de classes, isso é o que torna a modelagem de Fraude tão delicada, um evento que é raro e que acontece de forma sútil, vamos criar hipóteses para fazer o Feature engineering com esses dados, afim de maximizar as métricas: <b> ROC AUC | Precision | Recall </b>.

Tentar medir a performance do modelo com a Acurácia, seria um engano pois teriamos uma acurácia alta, o que de fato não resolveria o nosso problema, pois o conjunto possiu classes desbalanceadas, vamos focar nestas três métricas listadas acima, que não variam com o desbalanceamento das classes. 

<br>

### Limpeza de dados 

Antes de iniciar uma análise dos dados, vamos limpar os dados, não será necessário apagar transações duplicadas pois faz total sentido, conter transaçõe que são feitas rotineiramente.  

* Missing values 
* Ruídos 
* Tipo de dados

In [0]:
def missing_values(data):

    """ Resumo de dados nulos
        contidos no dataset """

    # total de nulos     
    missing = data.isnull().sum()
    total = missing.sort_values(ascending=True)
    
    # porcentagem 
    percent = (missing / len(data.index ) * 100).round(2).sort_values(ascending=True)

    # concatenação 
    table_missing = pd.concat([total, percent], axis=1, keys=['Números de NA', 'Porcentagem de NA'])

    return table_missing

In [0]:
missing_values(credit)

In [0]:
# Tipo de dados
credit.dtypes

<br>
<br>
<hr>


### Análise de dados

Nesta etapa vou tentar identificar alguns padrões, já tenho em mente que vou fazer alguns experimentos tratando os Outliers posteriormente, criar alguns modelos e comparar as aborgadens feitas. 

<hr>
<br>

Vou checar a distribuição de segundos, quero identificar o perído de tempo, em que uma transação fraudulenta é feita, comparando com as transações normais. 

In [0]:
# Comparar Distribuições

plt.figure(figsize=(16,12))

plt.subplot(2,1,1)
plt.title('Segundos Transação Normal')
sns.distplot(credit[credit['Class']==0]['Time'], color='purple')

print('\n')
print('\n')

plt.subplot(2,1,2)
plt.title('Segundos Transação Fraudulenta')
sns.distplot(credit[credit['Class']==1]['Time'], color='red')


Olhando para as duas Distribuições, conseguimos ver que o período de uma transação rotulada como Normal, ocorre em uma média de uma forma trivial o que é considerado algo normal, já as transações fraudulentas ocorrem em um período de tempo maior, acada <b> 50.000 segundos </b> uma nova transação fraudulenta é feita(<b> Em dois dias de transações </b>).É muito importante identificar que é uma distribuição muito próxima da <b> Uniforme </b> pois tem uma curva de densidade, quase no mesmo ângulo constante.


Precisamos verificar posteriormente, se ocorre uma transação fraudulenta mais de  uma vez, com os mesmos dados da transação. 

In [0]:
#  Fraudes repetidas ?

fraud = credit[credit['Class']==1].loc[credit.duplicated()]
print('Fraudes repetidas: {} '.format(len(fraud)))
print('\n')
fraud

<br>
Houve <b> 19 fraudes repetidas </b> neste dois dias, tem casos que houve três fraudes com o mesmo valor, o restante não passa de dois, algo que chama a atenção são os valores baixos das transações que houve um maior número de fraudes, talvez por que o fraudador considera-se o risco de ser pego comparado com o número de tentativas. 

<br>

In [0]:
# cmap
cmap = sns.diverging_palette(120, 40, sep=20, as_cmap=True, center='dark')

In [0]:
# Correlações
corr = credit.corr(method='pearson')

fig, ax = plt.subplots(figsize=(23,15))

# cmap=Greys

plt.title('Matriz de Correlação', fontsize=16)
print('\n')
correlacao = sns.heatmap(corr, annot=True, cmap='Blues', ax=ax, lw=3.3, linecolor='lightgray')
correlacao

Olhando as correlações de Pearson, conseguimos ver que não há correlações positivasgrandes,apenas algumas features passam de <b> 0.30 de correlação, que são <b> V7 com Amount | V20 com Amount </b>.

(Vamos olhar posteriormente as correlações) 

<br>

In [0]:
# Distribuições 

# definindo plot 
cols_names = credit.drop(['Class', 'Amount', 'Time'], axis=1)
idx = 0

# Separando classes
fraud = credit[credit['Class']==1]
normal = credit[credit['Class']==0]

# figura do plot 
fig, ax = plt.subplots(nrows=7, ncols=4, figsize=(18,18))
fig.subplots_adjust(hspace=1, wspace=1)

for col in cols_names:
    idx += 1
    plt.subplot(7, 4, idx)
    sns.kdeplot(fraud[col], label="Normal", color='blue', shade=True)
    sns.kdeplot(normal[col], label="Fraud", color='red', shade=True)
    plt.title(col, fontsize=11)
    plt.tight_layout()

Todas as distribuições das variáveis que possuem uma "máscara", não sabemos a representação real dessas variáveis pois elas foram ocultadas, mas através da distribuição com a Densidade de kernel, pode ver muito bem as curvas de cada uma, comparando com uma transação normal ou Fraude. 



Fraude: Algumas se aproximam de uma Distribuição Gaussiana, com uma pico maior e uma cauda um bem extensa, possivelmente temos uma variação maior dos dados, nestas features.

<br>

In [0]:
# Intervalor de valores das Fraudes 
credit[credit['Class']==1]['Amount'].value_counts

In [0]:
plt.figure(figsize=(14,8))
sns.boxplot(x='Class', y='Amount', data=credit, palette='coolwarm', order=credit.loc[300:320, 'Amount'])

In [0]:
# Qual o valor médio da transação fraudulenta ? 
print('Valor médio Fraude: {} | Valor médio Normal: {}'.format(credit[credit['Class']==1]['Amount'].mean() , credit[credit['Class']==0]['Amount'].mean()))

In [0]:
# Qual o maior valor de fraude ?
print('Maior valor Fraude: {}  | Maior valor Normal: {}'.format(credit[credit['Class']==1]['Amount'].max(), credit[credit['Class']==0]['Amount'].max()))

In [0]:
plt.figure(figsize=(12,6))
plt.title('Valor de Transações por Classe', fontsize=15)
sns.barplot(x='Class', y='Amount', data=credit, palette='GnBu')

In [0]:
# Distribuição de valores das Transações 
plt.figure(figsize=(10,5))
plt.title('Transações', fontsize=14)
plt.grid(False)
sns.kdeplot(credit['Amount'], color='lightblue', shade=True)

### Resumo de Análise

Após esta etapa de exploração dos dados, conseguimos ter insights bons sobre o conjunto, transações fraudulentas tendem a ter valores menores do que transações normais, fraudes repetidas contém valores menores, outro ponto que descobrimos é que o tempo, entre uma fraude e outra, como os dados nos dizem sobre apenas dois dias de transações, fica díficil inferirmos um valor pontual, em que o fraudador faz uma transferência, mas o tempo possiu uma variação maior que transações normais.

Vamos criar diversos experimentos, em busca de um modelo robusto e que atinja o nosso objetivo. 

<br>
<hr>
<br>

### Baseline 

Vou criar um modelo base, de forma que eu possa comparar os próximos resultados de outros experimentos, com este modelo puro e simples que vamos construir. 

In [0]:
# Separando feature| classe 

X = credit.drop('Class', axis=1)
y = credit['Class']


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=42)


# StandardScaler 
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [0]:
# Modelo Baseline

baseline = LogisticRegression(random_state=42)
baseline.fit(X_train, y_train)
y_baseline = baseline.predict(X_test)

# Probabilidades 
y_proba_baseline = baseline.predict_proba(X_test)[:,1]

print(classification_report(y_test, y_baseline))
print('\n')
print('AUC: {}%'.format(roc_auc_score(y_test, y_proba_baseline)))
print('Precision-Recall: {}'.format(average_precision_score(y_test, y_proba_baseline)))

<br>

### Baseline

* AUC: 0.97
* AUPRC: 0.78
* Precision: 0.88
* Recall: 0.63

Um modelo baseline puro nos trouxe uma idéia do quanto podemos melhorar o modelo base, para um modelo melhor, vamos criar alguns experimentos para obter uma performance melhor. 

<hr>
<br>

### Experimentação 

Vamos testar alguns algoritmos como: 

* Random Forest 
* SVM 

Vamos aplica-los separadamente, mas vamos tentar outras alternativas na modelagem, aplicar técnicas de balanceamento dos dados, e também validar nossos modelos apartir de Cross validation. 

<br>
<hr>

Vamos aplicar primeiro uma Random Forest, combinada com uma técnica de OverSampling que basicamente vai, criar dados sintéticos na class minonitária que é a classe fraude, vamos testar esta abordagem e ver a performance do modelo. Assim por diante, vamos aplicar UnderSampling também para medir a performance em uma outra abordagem. 


<p align=center>
<img src="https://blog.strands.com/hs-fs/hubfs/Screenshot%202019-07-18%20at%2014.15.15.png?width=600&name=Screenshot%202019-07-18%20at%2014.15.15.png" width="60%"></p>


<br>
<br>


### Random Forest 

Vamos construir um modelo Random Forest como <b> 200 árvores </b> combinado com a técnica de OverSampling <b> SMOTE </b> e ver os resultados das métricas que estamos buscando otimizar que são: 

* AUC 
* Precision
* Recall 
* AUPRC  

<hr>

In [0]:
X = credit.drop('Class', axis=1)
y = credit['Class']

# Validação 
KFold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


fold = 0
for train_index, test_index in KFold.split(X,y):
      fold += 1 
      print('Fold: ', fold)
      print('Treino: ',train_index.shape[0])
      print('Teste: ', test_index[0])

      # Split
      X = credit.drop('Class', axis=1)
      y = credit['Class']

      # OverSampling SMOTE 
      smote = SMOTE(random_state=42)
      X, y = smote.fit_sample(X, y)
      print('Normal: {}  |  Fraude: {}'.format(np.bincount(y)[0], np.bincount(y)[1]))

      # separando dados 
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index] 

      
      # pré-processamento 
      scaler = QuantileTransformer(random_state=42)
      X_train = scaler.fit_transform(X_train)
      X_test = scaler.transform(X_test)

      # Criando modelo 
      forest = RandomForestClassifier(n_estimators=200, max_depth=13, min_samples_split=9,
                                    random_state=42)
      forest.fit(X_train, y_train)
      y_pred_forest = forest.predict(X_test)
      y_proba_forest = forest.predict_proba(X_test)[:,1]


      # Métricas
      print('\n')
      print(classification_report(y_test, y_pred_forest))
      print('--------------'*5)
      print('\n')
      auc_forest = roc_auc_score(y_test, y_proba_forest)
      precision_forest = precision_score(y_test, y_pred_forest)
      recall_forest = recall_score(y_test, y_pred_forest)
      auprc_forest = average_precision_score(y_test, y_proba_forest)

In [0]:
# Validação Random Forest 
print('Random Forest')
print('\n')

print('AUC: ', np.mean(auc_forest))
print('Precision: ', np.mean(precision_forest))
print('Recall: ', np.mean(recall_forest))
print('Precision-Recall: ', np.mean(auprc_forest))


print('\n')
print('\n')

# Curva ROC random forest 
auc_forest = np.mean(auc_forest)
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_test, y_proba_forest)

# plot 
plt.figure(figsize=(12,7))
plt.plot(fpr_forest, tpr_forest, color='blue', label='AUC: {}'.format(auc_forest))
plt.fill_between(fpr_forest, tpr_forest, color='skyblue', alpha=0.3)
plt.plot([0,1], [0,1], color='black', ls='--', label='Reference line')
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('ROC Random Forest', fontsize=16)
plt.legend(loc=4, fontsize=14)
plt.grid(False)
plt.show()

In [0]:
# Precision-Recall Random Forest 

plt.figure(figsize=(10,5))
viz = PrecisionRecallCurve(forest)
viz.fit(X_train, y_train)
viz.score(X_test, y_test)

<hr>

### Resumo Random Forest 

O modelo com <b> 200 árvores </b>, combinado com um método de pré-processamento chamado <b> QuantileTransformer </b> que tem o objetivo, de reduzir o impacto de posssíveis Outliers nos dados, ele vai aproximar a distriuição das features através do IQR (intervalo interquartil), também fiz uso da técnica <b> SMOTE </b> que cria dados sintéticos, da classe minonitária igualando com a classe majoritária. 

<br>

Em resumo o modelo não alcançou um resultado satisfatório, o AUC do modelo ficou alto, na validação o resultado foi de <b> 97%  </b>, mas as outras métricas não atingiram um valor esperado. Precision ficou alta mas o Recall não chegou a subir muito comparado a baseline, o trade-off Precision-Recall é a métrica principal em nossa avaliação, e díficil de otimizar, pois queremos um modelo que consiga separar bem uma fraude de uma transação normal, e também que identifique bem uma fraude quando ela realmente ocorrer. 





<hr>
<br>
<hr>
<br>

### SVM 

Vamos aplicar o SVM que é outro algoritmo robusto, vamos combinar nele um outro pré-processamento nos dados, vamos utilizar o UnderSampling agora, onde vamos diminuir a classe majoritária, na tentativa de igualar as classes.

SVM tem a função de kernel, que permite construir um Hiperplano que melhor se adeque aos dados. 

<br>
<hr>

In [0]:
# SVM 
X = credit.drop('Class', axis=1)
y = credit['Class']

# UnderSampling  
under = NearMiss()
X, y = under.fit_sample(X, y)


# Validação 
KFold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


fold = 0
for train_index, test_index in KFold.split(X,y):
      fold += 1 
      print('Fold: ', fold)
      print('Treino: ', train_index.shape[0])
      print('Teste: ', test_index[0])

      # Classe balanceadas
      print('Normal: {}  |  Fraude {}'.format(np.bincount(y)[0], np.bincount(y)[1]))

      # separando dados 
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index] 

      
      # pré-processamento 
      scaler = RobustScaler()
      X_train = scaler.fit_transform(X_train)
      X_test = scaler.transform(X_test)

      # Criando modelo 
      svm = SVC(C=1.0, gamma=0.5, random_state=42, probability=True)
      svm.fit(X_train, y_train)
      y_pred_svm = svm.predict(X_test)
      y_proba_svm = svm.predict_proba(X_test)[:,1]


      # Métricas
      print('\n')
      print(classification_report(y_test, y_pred_svm))
      print('-------------'*5)
      print('\n')
      auc_svm = roc_auc_score(y_test, y_proba_svm)
      precision_svm = precision_score(y_test, y_pred_svm)
      recall_svm = recall_score(y_test, y_pred_svm)
      auprc_svm = average_precision_score(y_test, y_proba_svm)

In [0]:
# Validação SVM 
print('SVM')
print('\n')

print('AUC: ', np.mean(auc_svm))
print('Precision: ', np.mean(precision_svm))
print('Recall: ', np.mean(recall_svm))
print('Precision-Recall: ', np.mean(auprc_svm))


print('\n')
print('\n')

# Curva ROC random forest 
auc_svm = np.mean(auc_svm)
fpr_svm, tpr_svm, thresholds_svm = roc_curve(y_test, y_proba_svm)

# plot 
plt.figure(figsize=(12,7))
plt.plot(fpr_svm, tpr_svm, color='blue', label='AUC: {}'.format(auc_svm))
plt.fill_between(fpr_svm, tpr_svm, color='skyblue', alpha=0.3)
plt.plot([0,1], [0,1], color='black', ls='--', label='Reference line')
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('ROC SVM', fontsize=16)
plt.legend(loc=4, fontsize=14)
plt.grid(False)
plt.show()

In [0]:
# Precision-Recall SVM 

plt.figure(figsize=(10,5))
plt.title('Precision-Recall', fontsize=16)
viz = PrecisionRecallCurve(svm)
viz.fit(X_train, y_train)
viz.score(X_test, y_test)

<hr>
<br>

### Resumo SVM 

Com SVM conseguimos melhorar nosso Recall para: <b> 95% </b>, consequentemente nossa Precision caiu, mas em geral o desempenho deste modelo foi superior a Random Forest, AUC foi a única com um descréscimom, Precision-Recall: <b> 96% </b> foi um resultado muito bom. Combinando o SVM com  a técnica de UnderSampling obtivemos resultados bons neste experimento.

Vou criar mais dois modelos utilizando dois algoritmos de <b> Gradient Boosting </b> para ver se conseguimos, resultados ainda melhores que o SVM. 




<hr>
<br>
<hr>
<br>
<br>

### XGboost 

O XGBoost é um algoritmo de Machine learning, baseado em árvore de decisão e que utiliza uma estrutura de Gradient boosting.

Em problemas de previsão envolvendo dados não estruturados, como imagens, textos e vídeos, as redes neurais artificiais tendem a superar todos os outros algoritmos ou frameworks.

No entanto, quando se trata de dados estruturados/tabulares, algoritmos baseados em árvore de decisão são considerados os melhores da sua classe no momento.

Vamos criar um modelo com o XGBoost. 


(Em um próximo projeto, vou explicar o funcionamento do XGboost por inteiro) 

<p align=center>
<img src="https://pythonawesome.com/content/images/2018/06/xgboost.png" width="60%"></p>



<br>
<hr>

### XGboost + SMOTE   

Vamos aplicar agora a técnica de OverSampling com o XGboost, vou aderir um experimento com o <b> SMOTE </b> que será responsável por aplicar o OverSampling, vou igualar as classes maximizando as proporções da classe 1.

<br>

In [0]:
X = credit.drop('Class', axis=1)
y = credit['Class']


# Validação 
KFold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
 
precision_xgboost = []
recall_xgboost = []
auc_xgboost = []
precision_recall_xgboost = []


fold = 0
for train_index, test_index in KFold.split(X,y):
      fold += 1 
      print('Fold: ', fold)
      print('Treino: ',train_index.shape[0])
      print('Teste: ', test_index[0])

      # OverSampling SMOTE 
      smt = SMOTE(random_state=42)
      X, y = smt.fit_sample(X, y)
      print('Normal: {}  |  Fraude: {}'.format(np.bincount(y)[0], np.bincount(y)[1]))

      # separando dados 
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index] 

      
      # pré-processamento 
      scaler = QuantileTransformer()
      X_train = scaler.fit_transform(X_train)
      X_test = scaler.transform(X_test)

      # XGboost 
      xgb = XGBClassifier(n_estimators=300, max_delta_step=1 ,eval_metric='aucpr', 
                          cpu_history='gpu', random_state=42)
      xgb.fit(X_train, y_train)
      y_pred = xgb.predict(X_test)
  

      # Métricas
      precision_recall_xgboost = average_precision_score(y_test, y_pred)
      precision_xgboost = precision_score(y_test, y_pred)
      recall_xgboost = recall_score(y_test, y_pred)
      auc_xgboost  = roc_auc_score(y_test, y_pred)
      print('Precision-Recall: ', average_precision_score(y_test, y_pred))
      print('\n')
      print('\n')



# Validação final  
print('Precision-Recall: ', np.mean(precision_recall_xgboost))
print('Recall: ', np.mean(recall_xgboost))
print('Precision: ', np.mean(precision_xgboost))
print('AUC: ', np.mean(auc_xgboost))

<hr>
<br>
<br>

In [0]:
# Validação XGboost + SMOTE
print('XGboost')
print('\n')

print('AUC: ', np.mean(auc_xgboost))
print('Precision: ', np.mean(precision_xgboost))
print('Recall: ', np.mean(recall_xgboost))
print('Precision-Recall: ', np.mean(precision_recall_xgboost))


print('\n')
print('\n')

# Curva ROC random forest 
roc_auc_xgboost = np.mean(auc_xgboost)
fpr_xgboost, tpr_xgboost, thresholds_xgboost = roc_curve(y_test, y_pred)

# plot 
plt.figure(figsize=(12,7))
plt.plot(fpr_xgboost, tpr_xgboost, color='blue', label='AUC: {}'.format(roc_auc_xgboost))
plt.fill_between(fpr_xgboost, tpr_xgboost, color='skyblue', alpha=0.3)
plt.plot([0,1], [0,1], color='black', ls='--', label='Reference line')
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('ROC XGboost', fontsize=16)
plt.legend(loc=4, fontsize=14)
plt.grid(False)
plt.show()

<hr>
<br>
<br>

In [0]:
# Precision-Recall XGboost + SMOTE 

plt.figure(figsize=(10,5))
plt.title('Precision-Recall', fontsize=16)
viz = PrecisionRecallCurve(XGBClassifier(n_estimators=300, max_delta_step=1 ,eval_metric='aucpr', 
                          cpu_history='gpu', random_state=42))
viz.fit(X_train, y_train)
viz.score(X_test, y_test)

<hr>
<br>

### XGboost + NearMiss 

Vamos combinar o XGboost com a técnica de UnderSampling NearMiss, vou utilizar os mesmos parâmetros que o modelo anterior, neste caso vamos minimizar a classe majoritária a uma prorção igual nossa classe minonitária. 



In [0]:
X = credit.drop('Class', axis=1)
y = credit['Class']

# UnderSampling NearMiss
under = NearMiss(random_state=42)
X,y = under.fit_sample(X, y)

# Validação 
KFold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

resultados = []
fold = 0
for train_index, test_index in KFold.split(X,y):
      fold += 1 
      print('Fold: ', fold)
      print('Treino: ',train_index.shape[0])
      print('Teste: ', test_index[0])

      # Classes balanceadas 
      print('Normal: {} | Fraude: {}'.format(np.bincount(y)[0], np.bincount(y)[1]))


      # separando dados 
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index] 

      
      # pré-processamento 
      scaler = StandardScaler()
      X_train = scaler.fit_transform(X_train)
      X_test = scaler.transform(X_test)

      # XGboost 
      xgb = XGBClassifier(n_estimators=300, max_delta_step=1 ,eval_metric='aucpr', 
                          cpu_history='gpu', random_state=42)
      xgb.fit(X_train, y_train)
      y_pred = xgb.predict(X_test)

    # Métricas
      precision_recall_xgboost = average_precision_score(y_test, y_pred)
      precision_xgboost = precision_score(y_test, y_pred)
      recall_xgboost = recall_score(y_test, y_pred)
      auc_xgboost  = roc_auc_score(y_test, y_pred)
      print('Precision-Recall: ', average_precision_score(y_test, y_pred))
      print('\n')
      print('\n')



# Validação final  
print('Precision-Recall: ', np.mean(precision_recall_xgboost))
print('Recall: ', np.mean(recall_xgboost))
print('Precision: ', np.mean(precision_xgboost))
print('AUC: ', np.mean(auc_xgboost))

<hr>
<br>
<hr>
<br>

In [0]:
# Validação XGboost + NearMiss  
print('XGboost')
print('\n')

print('AUC: ', np.mean(auc_xgboost))
print('Precision: ', np.mean(precision_xgboost))
print('Recall: ', np.mean(recall_xgboost))
print('Precision-Recall: ', np.mean(precision_recall_xgboost))


print('\n')
print('\n')

# Curva ROC random forest 
roc_auc_xgboost = np.mean(auc_xgboost)
fpr_xgboost, tpr_xgboost, thresholds_xgboost = roc_curve(y_test, y_pred)

# plot 
plt.figure(figsize=(12,7))
plt.plot(fpr_xgboost, tpr_xgboost, color='blue', label='AUC: {}'.format(roc_auc_xgboost))
plt.fill_between(fpr_xgboost, tpr_xgboost, color='skyblue', alpha=0.3)
plt.plot([0,1], [0,1], color='black', ls='--', label='Reference line')
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('ROC XGboost', fontsize=16)
plt.legend(loc=4, fontsize=14)
plt.grid(False)
plt.show()

<br>
<br>
<hr>
<br>

In [0]:
# Precision-Recall XGboost + NearMiss 

plt.figure(figsize=(10,5))
plt.title('Precision-Recall', fontsize=16)
viz = PrecisionRecallCurve(XGBClassifier(n_estimators=300, max_delta_step=1 ,eval_metric='aucpr', 
                          cpu_history='gpu', random_state=42))
viz.fit(X_train, y_train)
viz.score(X_test, y_test)

<hr>
<br>

### XGboost Resumo 

A melhor combinação até agora foi <b> XGboost + UnderSampling(NearMiss) </b> conseguimos obter valores muito bons, reduzimos nossa classe majoritária, balanceando as classes o algoritmo conseguiu generalizar melhor os dados, com uma AUPRC de <b> 94% </b> na validação, foi um resultado excelente do modelo, os dados sintéticos que o SMOTE acaba gerando, acaba não fazendo muito sentido para o algoritmo, pois são "dados repetidos" e que mostra, pouquíssima generalização de Fraudes. 



<br>
<hr>
<br>
<hr>
<br>

### LightGBM 


<hr>
<br>

<p align=center>
<img src="https://blog.mockun.com/content/images/2018/09/LightGBM.png" width="60%"></p>

##### em construção................